# 커리큘럼 preview quiz 검수

In [1]:
import pandas as pd
import re
import json
import random
import csv
from tqdm.notebook import tqdm
from collections import Counter

In [135]:
df_mid = pd.read_csv("DB_MID.csv")
df_elem = pd.read_csv("DB_Elem.csv")
df_high =pd.read_csv("Curriculum_High.csv")

In [3]:
df_mid.columns

Index(['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분', 'title', 'new_DES',
       'DESCRIPTION', 'Key Expression (영어)', 'Key Expression (한국어)',
       'Sub Expression (영어)', 'Sub Expression (한국어)', '학습목표', 'Quiz 1',
       'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3', 'Answer 1',
       'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 30', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34'],
      dtype='object')

In [165]:
def preview_quiz_check_valid(data:pd.DataFrame):
    # 에러 유무 확인
    error_found = False
    
    # 변수 가져오기
    for i in tqdm(range(len(data)), total = len(data)):
        # quiz 1
        quiz_1 = data["Quiz 1"][i]
        korean_1 = data["Korean 1"][i]
        option_1_1 = data["Option 1-1"][i]
        option_1_2 = data["Option 1-2"][i]
        option_1_3 = data["Option 1-3"][i]
        answer_1 = data["Answer 1"][i]
        index_1 = data["Index 1"][i]

        # quiz 2
        quiz_2 = data["Quiz 2"][i]
        korean_2 = data["Korean 2"][i]
        option_2_1 = data["Option 2-1"][i]
        option_2_2 = data["Option 2-2"][i]
        option_2_3 = data["Option 2-3"][i]
        answer_2 = data["Answer 2"][i]
        index_2 = data["Index 2"][i]

        # QUIZ 1 확인
        # 필드값 누락 체크
        if (    not quiz_1
                or not korean_1
                or not option_1_1
                or not option_1_2
                or not option_1_3
                or not answer_1
                or not index_1
            ):
            print(f"\nQuiz 1 {i+2}번째 행 : 필드값 누락 체크")
            error_found = True

        # 1. 빈칸이 하나 이상있고, 두 개 이상이 없는지 체크
        appropriate_blank = r"_+"
        underscores = re.findall(appropriate_blank, quiz_1)

        if len(underscores) > 1:
            print(f"\nQuiz 1 {i+2}번째 행 : Blank error - 밑줄이 두 개 이상 존재")
            error_found = True
        elif len(underscores) ==1:
            pass
        else:
            print(f"\nQuiz 1 {i+2}번째 행 : Blank error - 밑줄이 존재하지 않음")
            error_found = True

        # 1-1 한국어 해석에 빈칸이 있는지
        appropriate_blank_korean = r'[_~]+'
        korean_underscores = re.findall(appropriate_blank_korean, korean_1)
        if len(korean_underscores) >= 1:
            print(f"\nQuiz 1 {i+2}번째 행 : 한국어 해석에 밑줄 있음")
            error_found = True

        # 2. 정답이 옵션에 포함되어 있는지 체크
        options_1 = [option_1_1, option_1_2, option_1_3]

        # answer가 options에 있는지 확인
        if answer_1 not in options_1:
            print(f"\nQuiz 1 {i+2}번째 행 : Answer not in options error")
            error_found = True

        # answer가 숫자로 이루어져있는지 확인
        if isinstance(index_1, str) and not index_1.isnumeric():
            print(f"\nQuiz 1 {i+2}번째 행 : Answer in options error")
            error_found = True

        # 3. 정답의 인덱스가 answer_index와 일치하는지 확인
        index_1 = int(index_1)
        if options_1[index_1 - 1] != answer_1:
            print((f"\nQuiz 1 {i+2}번째 행 : 정답의 인덱스가 answer_index와 불일치"))
            #print(f"{options_1}의 정답이 {answer_1}이지만 index {index_1}로 적혀있는 에러")
            error_found = True

        # QUIZ 2 확인
        # 필드값 누락 체크
        if (    not quiz_2
                or not korean_2
                or not option_2_1
                or not option_2_2
                or not option_2_3
                or not answer_2
                or not index_2
            ):
            print(f"\nQuiz 2 {i+2}번째 행 : 필드값 누락 체크")
            error_found = True

        # 1. 빈칸이 하나 이상있고, 두 개 이상이 없는지 체크
        appropriate_blank = r"_+"
        underscores = re.findall(appropriate_blank, quiz_2)

        if len(underscores) > 1:
            print(f"\nQuiz 2 {i+2}번째 행 : Blank error - 밑줄이 두 개 이상 존재")
            error_found = True
        elif len(underscores) ==1:
            pass
        else:
            print(f"\nQuiz 2 {i+2}번째 행 : Blank error - 밑줄이 존재하지 않음")
            error_found = True
        
        # 1-1 한국어 해석에 빈칸이 있는지
        appropriate_blank_korean = r'[_~]+'
        korean_underscores = re.findall(appropriate_blank_korean, korean_2)
        if len(korean_underscores) >= 1:
            print(f"\nQuiz 2 {i+2}번째 행 : 한국어 해석에 밑줄 있음")
            error_found = True

        # 2. 정답이 옵션에 포함되어 있는지 체크
        options_2 = [option_2_1, option_2_2, option_2_3]

        # answer가 options에 있는지 확인
        if answer_2 not in options_2:
            print(f"\nQuiz 2 {i+2}번째 행 : Answer not in options error")
            error_found = True

        # answer가 숫자로 이루어져있는지 확인
        if isinstance(index_2, str) and not index_2.isnumeric():
            print(f"\nQuiz 2 {i+2}번째 행 : Answer in options error")
            error_found = True

        # 3. 정답의 인덱스가 answer_index와 일치하는지 확인
        index_2 = int(index_2)
        if options_2[index_2 - 1] != answer_2:
            print((f"\nQuiz 2 {i+2}번째 행 : 정답의 인덱스가 answer_index와 불일치"))
            #print(f"{options_2}의 정답이 {answer_2}이지만 index {index_2}로 적혀있는 에러")
            error_found = True
    
    if not error_found:
        print("모든 데이터가 정상입니다다")


In [158]:
df_mid['Korean 1'][:5]

0    A: 내 이름은 Felix야. 나는 Australia 출신이야. B: 만나서 반가워.
1                             네가 가장 좋아하는 스포츠는 무엇이에요?
2                 A: 너는 휴일에 무엇을 할 예정이니? B: 여행을 갈 거야.
3                                우리 도시에는 많은 공원이 있어요.
4                  A: 너는 ______ 해야 해. B: 알겠어, 노력해볼게.
Name: Korean 1, dtype: object

In [167]:
preview_quiz_check_valid(df_mid)

100%|██████████| 1110/1110 [00:00<00:00, 14620.82it/s]


Quiz 1 6번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 6번째 행 : 한국어 해석에 밑줄 있음

Quiz 1 65번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 65번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 76번째 행 : 한국어 해석에 밑줄 있음

Quiz 1 108번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 108번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 127번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 262번째 행 : 한국어 해석에 밑줄 있음

Quiz 1 500번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 500번째 행 : 한국어 해석에 밑줄 있음

Quiz 1 545번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 545번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 834번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 933번째 행 : 한국어 해석에 밑줄 있음


In [163]:
preview_quiz_check_valid(df_elem)

100%|██████████| 769/769 [00:00<00:00, 14904.21it/s]


Quiz 2 44번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 58번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 107번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 123번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 126번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 132번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 208번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 252번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 253번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 286번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 297번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 303번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 326번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 392번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 424번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 449번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 459번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 468번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 495번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 523번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 545번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 572번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 573번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 584번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 608번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 616번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 660번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 689번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 731번째 행 : 한국어 해석에 밑줄 있음

Quiz 2 733번째 행 : 한국어 해석에 밑줄 있음


In [164]:
# preview_quiz_check_valid(df_high)

## 중복 정답 여부 확인

In [63]:
from openai import OpenAI
from getpass import getpass
from jinja2 import Template
from openai.lib._parsing import type_to_response_format_param
from pydantic import BaseModel
import json
import pandas as pd
from tqdm import tqdm

In [64]:
api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [108]:
prompt_template = Template("""
[PURPOSE]
You are going to solve the quiz and write down the answer_reason.
There is not necessarily a single correct answer.
Select all that are correct.
                           
[Example]
```
Input: 
    {
    "quiz": "What is your ______ sport?",
    "korean": "네가 가장 좋아하는 스포츠는 무엇이에요?",
    "option_1": "like",
    "option_2": "favorite",
    "option_3": "best",
  }

Output:
    {   
    "possible_answer": ["favorite"],
    "answer_reason": "The word 'favorite' is the correct answer because the sentence is asking about someone's most preferred sport.

1. 'favorite' means 가장 좋아하는 in Korean, which matches the meaning of the given Korean sentence.
2. 'like' is a verb (좋아하다) and does not fit grammatically in this sentence.
3. 'best' means 최고의, which refers to the highest quality rather than personal preference.

Thus, 'favorite' is the correct choice."
}
```                     
```
Input: 
    {
    "quiz": "A: What are you _____ on your holiday? B: I’m going to travel.",
    "korean": "A: 너는 휴일에 무엇을 할 예정이니? B: 여행을 갈 거야.",
    "option_1": "planning to do",
    "option_2": "going to do",
    "option_3": "will do",
  }

Output:
    {   
    "possible_answer": ["planning to do, going to do",]
    "answer_reason": "Both 'planning to do' and 'going to do' fit naturally in the sentence.

1. 'Going to do' is commonly used for future plans and matches the response "I’m going to travel."
2. 'Planning to do' also makes sense because it refers to making arrangements for the holiday.
3. 'Will do' is less natural in this context because it’s not commonly used for asking about planned activities."

Thus, 'planning to do' and 'going to do' are the correct choice."
}
```

[Input]
```
QUIZ : {{quiz}}
KOREAN : {{korean}}
OPTION_1 : {{option_1}}
OPTION_2 : {{option_2}}
OPTION_3 : {{option_3}}
```
""")

In [109]:
class Reason(BaseModel):
    possible_answer : list[str]
    answer_reason : str

In [102]:
response_format = type_to_response_format_param(Reason)

In [103]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'properties': {'possible_answer': {'items': {'type': 'string'},
     'title': 'Possible Answer',
     'type': 'array'},
    'answer_reason': {'title': 'Answer Reason', 'type': 'string'}},
   'required': ['possible_answer', 'answer_reason'],
   'title': 'Reason',
   'type': 'object',
   'additionalProperties': False},
  'name': 'Reason',
  'strict': True}}

In [69]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You are going to solve the quiz and write down the answer_reason."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = Reason,
    )
    return response.choices[0].message.parsed

In [71]:
prompt = prompt_template.render(
    quiz = "___ I borrow your book?",
    korean = "내 책 좀 빌려도 될까요?",
    option_1 = "May",
    option_2 = "Can",
    option_3 = "Might"
)

In [72]:
response = completion(prompt)

In [74]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent=2)
print(response_output)

{
  "possible_answer": [
    "May",
    "Can"
  ],
  "answer_reason": "The sentence is asking for permission, as indicated by the Korean '내 책 좀 빌려도 될까요?' which translates to 'May I borrow your book?'.\n\n1. 'May' is the traditional modal verb used for asking permission in formal or polite contexts.\n2. 'Can' is commonly used in everyday conversation to ask for permission, even though its primary meaning is about ability.\n3. 'Might' is less commonly used in this context and tends to sound overly formal or archaic, making it less suitable compared to 'May' and 'Can'.\n\nThus, 'May' and 'Can' are both acceptable answers."
}


In [75]:
df_mid.columns

Index(['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분', 'title', 'new_DES',
       'DESCRIPTION', 'Key Expression (영어)', 'Key Expression (한국어)',
       'Sub Expression (영어)', 'Sub Expression (한국어)', '학습목표', 'Quiz 1',
       'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3', 'Answer 1',
       'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 30', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34'],
      dtype='object')

In [ ]:
def answer_duplication_check_quiz1(data:pd.DataFrame, output_filename = "PreviewQuiz1_MID_duplicate_answer.csv"):
    # 에러 리스트 확인
    duplicate_data = []

    # 파일 변환
    for i in tqdm(range(len(data)), total=len(data)):
        prompt = prompt_template.render(
            quiz = data.loc[i, 'Quiz 1'],
            korean = data.loc[i, 'Korean 1'],
            option_1 = data.loc[i, 'Option 1-1'],
            option_2 = data.loc[i, 'Option 1-2'],
            option_3 = data.loc[i, 'Option 1-3']
        )

        response = client.beta.chat.completions.parse(
            model = "o3-mini",
            reasoning_effort = "low",
            messages = [
                {"role" : "system", "content" : "You are going to solve the quiz and write down the answer_reason."},
                {"role" : "user", "content" : prompt}
            ],
            response_format = Reason
        )

        # 응답 결과 출력 및 결과 출력력
        output = response.choices[0].message.parsed
        possible_answers = output.possible_answer
        answer_reason = output.answer_reason

        # 중복 답변 체크
        if len(possible_answers) > 1:
            row_number = i+2
            duplicate_data.append({"Row": row_number, "Reason": answer_reason})

    # 최종적으로 중복된 문제를 출력
    if duplicate_data:
        df_duplicates = pd.DataFrame(duplicate_data)
        df_duplicates.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\n중복된 문제 목록이 '{output_filename}' 파일에 저장되었습니다.")
    else:
        print("\n중복된 문제가 없습니다.")
    

In [130]:
def answer_duplication_check_quiz2(data:pd.DataFrame,output_filename = "PreviewQuiz2_MID_duplicate_answer.csv"):
    # 에러 리스트 확인
    duplicate_data = []

    # 파일 변환
    for i in tqdm(range(len(data)), total=len(data)):
        prompt = prompt_template.render(
            quiz = data.loc[i, 'Quiz 2'],
            korean = data.loc[i, 'Korean 2'],
            option_1 = data.loc[i, 'Option 2-1'],
            option_2 = data.loc[i, 'Option 2-2'],
            option_3 = data.loc[i, 'Option 2-3']
        )

        response = client.beta.chat.completions.parse(
            model = "o3-mini",
            reasoning_effort = "low",
            messages = [
                {"role" : "system", "content" : "You are going to solve the quiz and write down the answer_reason."},
                {"role" : "user", "content" : prompt}
            ],
            response_format = Reason
        )

        # 응답 결과 출력 및 결과 출력력
        output = response.choices[0].message.parsed
        possible_answers = output.possible_answer
        answer_reason = output.answer_reason

        # 중복 답변 체크
        if len(possible_answers) > 1:
            row_number = i+2
            duplicate_data.append({"Row": row_number, "Reason": answer_reason})

    # 최종적으로 중복된 문제를 출력
    if duplicate_data:
        df_duplicates = pd.DataFrame(duplicate_data)
        df_duplicates.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"\n중복된 문제 목록이 '{output_filename}' 파일에 저장되었습니다.")
    else:
        print("\n중복된 문제가 없습니다.")
    

In [128]:
sample = df_mid[:5]

In [129]:
answer_duplication_check_quiz1(sample)

100%|██████████| 5/5 [00:16<00:00,  3.28s/it]


중복된 문제 목록이 'PreviewQuiz_MID_duplicate_answer.csv' 파일에 저장되었습니다.


In [131]:
answer_duplication_check_quiz2(sample)

100%|██████████| 5/5 [00:18<00:00,  3.71s/it]


중복된 문제가 없습니다.


# 엑셀 전처리

In [184]:
import pandas as pd
db_mid = pd.read_csv("DB_MID.csv")
db_mid.columns

Index(['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분', 'title', 'new_DES',
       'DESCRIPTION', 'Key Expression (영어)', 'Key Expression (한국어)',
       'Sub Expression (영어)', 'Sub Expression (한국어)', '학습목표', 'Quiz 1',
       'Korean 1', 'Option 1-1', 'Option 1-2', 'Option 1-3', 'Answer 1',
       'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 30', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34'],
      dtype='object')

In [185]:
drop_col = ['textbook ID', '학년', '출판사', '교육과정', 'Lesson', '구분', 'title', 'new_DES',
       'DESCRIPTION', 'Key Expression (영어)', 'Key Expression (한국어)',
       'Sub Expression (영어)', 'Sub Expression (한국어)', '학습목표', 'Answer 1',
       'Index 1', 'Quiz 2', 'Korean 2', 'Option 2-1', 'Option 2-2',
       'Option 2-3', 'Answer 2', 'Index 2', '샘플 스크립트 (영어)', '샘플 스크립트 (한국어)',
       'Unnamed: 30', 'STRUCTURE - Title, New Desc 작업용', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34']
db_mid.drop(columns=drop_col, inplace=True)

In [186]:
db_mid = db_mid.rename(columns={
    'Quiz 1' : "quiz",
    'Korean 1' : "korean",
    'Option 1-1' : "option_1",
    'Option 1-2' : "option_2",
    'Option 1-3' : "option_3"
})
db_mid.head(3)

,quiz,korean,option_1,option_2,option_3
0,A: My name is Felix. I’m from ____.,A: 내 이름은 Felix야. 나는 Australia 출신이야. B: 만나서 반가워.,America,Australia,Canada
1,What is your ______ sport?,네가 가장 좋아하는 스포츠는 무엇이에요?,like,favorite,best
2,A: What are you _____ on your holiday? B: I’m ...,A: 너는 휴일에 무엇을 할 예정이니? B: 여행을 갈 거야.,planning to do,going to do,will do


In [192]:
df_mid = pd.DataFrame(df_mid)

In [193]:
# df_mid.index = range(1, len(df_mid))
df_mid.index = range(1, len(df_mid) + 1)
df_mid.index.name = 'id'


db_mid[:100].to_csv("test1.csv", encoding="utf-8-sig", index=True)